In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Load cleaned data
df = pd.read_csv('../data/processed/loan_default_cleaned.csv')

# Rename Default column to avoid SQL reserved keyword conflict
df = df.rename(columns={'Default': 'is_default'})

# Create SQLite database
conn = sqlite3.connect('../data/loan_default.db')
df.to_sql('loans', conn, if_exists='replace', index=False)
print("Database created successfully")

# ---- QUERY 1: Default rate by employment type ----
query1 = """
SELECT 
    EmploymentType,
    COUNT(*) as total_loans,
    SUM(is_default) as defaults,
    ROUND(AVG(is_default) * 100, 2) as default_rate_pct,
    ROUND(AVG(Income), 2) as avg_income,
    ROUND(AVG(LoanAmount), 2) as avg_loan_amount
FROM loans
GROUP BY EmploymentType
ORDER BY default_rate_pct DESC
"""
df_q1 = pd.read_sql_query(query1, conn)
print("\nQuery 1 — Default Rate by Employment Type:")
print(df_q1)

# ---- QUERY 2: Risk segmentation by credit score ----
query2 = """
SELECT 
    CASE 
        WHEN CreditScore < 400 THEN 'Very Poor (< 400)'
        WHEN CreditScore BETWEEN 400 AND 550 THEN 'Poor (400-550)'
        WHEN CreditScore BETWEEN 551 AND 700 THEN 'Fair (551-700)'
        WHEN CreditScore BETWEEN 701 AND 850 THEN 'Good (701-850)'
        ELSE 'Excellent (> 850)'
    END as credit_tier,
    COUNT(*) as total_loans,
    SUM(is_default) as defaults,
    ROUND(AVG(is_default) * 100, 2) as default_rate_pct,
    ROUND(AVG(InterestRate), 2) as avg_interest_rate,
    ROUND(AVG(LoanAmount), 2) as avg_loan_amount
FROM loans
GROUP BY credit_tier
ORDER BY default_rate_pct DESC
"""
df_q2 = pd.read_sql_query(query2, conn)
print("\nQuery 2 — Risk Segmentation by Credit Score:")
print(df_q2)

# ---- QUERY 3: Revenue at risk ----
query3 = """
SELECT 
    LoanPurpose,
    EmploymentType,
    COUNT(*) as defaulted_loans,
    ROUND(AVG(LoanAmount), 2) as avg_loan_amount,
    ROUND(SUM(LoanAmount), 2) as total_amount_at_risk
FROM loans
WHERE is_default = 1
GROUP BY LoanPurpose, EmploymentType
ORDER BY total_amount_at_risk DESC
LIMIT 10
"""
df_q3 = pd.read_sql_query(query3, conn)
print("\nQuery 3 — Top Revenue at Risk Segments:")
print(df_q3)

# ---- QUERY 4: Rolling default rate by age group ----
query4 = """
WITH age_stats AS (
    SELECT 
        Age,
        COUNT(*) as total,
        SUM(is_default) as defaults,
        ROUND(AVG(is_default) * 100, 2) as default_rate
    FROM loans
    GROUP BY Age
)
SELECT 
    Age,
    total,
    defaults,
    default_rate,
    ROUND(AVG(default_rate) OVER (
        ORDER BY Age
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ), 2) as rolling_avg_default_rate
FROM age_stats
ORDER BY Age
"""
df_q4 = pd.read_sql_query(query4, conn)
print("\nQuery 4 — Rolling Default Rate by Age:")
print(df_q4.head(20))

conn.close()

Database created successfully

Query 1 — Default Rate by Employment Type:
  EmploymentType  total_loans  defaults  default_rate_pct  avg_income  \
0     Unemployed        63824      8650             13.55    82272.37   
1      Part-time        64161      7677             11.97    82389.36   
2  Self-employed        63706      7302             11.46    82446.71   
3      Full-time        63656      6024              9.46    82890.30   

   avg_loan_amount  
0        128079.96  
1        127329.70  
2        127436.70  
3        127469.87  

Query 2 — Risk Segmentation by Credit Score:
         credit_tier  total_loans  defaults  default_rate_pct  \
0  Very Poor (< 400)        46654      6223             13.34   
1     Poor (400-550)        70120      8470             12.08   
2     Fair (551-700)        69627      7922             11.38   
3     Good (701-850)        68946      7038             10.21   

   avg_interest_rate  avg_loan_amount  
0              13.51        127181.81  
1  